### Ingest Module Example Usage

- Make sure this notebook has the kernel selected corresponding to your poetry environment.
- Poetry should handle loading the environment variables from the `.env` without having to install `python-dotenv`

In [1]:
import os
from rentradar.ingest.rentcast_client import RentCastAPIClient

RENTCAST_API_KEY = os.environ.get('RENTCAST_API_KEY')

query_params = {
    "city": "Charlottesville",
    "state": "VA"
}

The `RentCastAPIClient` object has a classmethod named `create` that pulls the desired data and stores it in an instance of the dataclass. The client also have a method named `to_frame` that will output the data as a pandas dataframe. The query params take in a city and state. Beware that the create endpoint has pagination, so it will keep pulling until all data for a city has been retrived. 

**THIS ISN'T FREE. MAKE SURE TO VIEW THE RENTCAST API DOCS**

### Long Term Rentals

- The following cell results in 3,593 long term rental properties in Charlottesville and it took 8 RentCast API requests to pull it all.
- API docs: https://developers.rentcast.io/reference/rental-listings-long-term

In [2]:
long_term_rentals_endpoint = "/listings/rental/long-term"

ltr = RentCastAPIClient.create(
    endpoint=long_term_rentals_endpoint, 
    query_params=query_params, 
    api_key=RENTCAST_API_KEY, 
    limit=500
)

ltr_df = ltr.to_frame()

INFO:rentradar.ingest.rentcast_client:Starting data fetch for endpoint: /listings/rental/long-term
INFO:rentradar.ingest.rentcast_client:Completed fetching all data for endpoint: /listings/rental/long-term


In [3]:
ltr_df.describe()

,latitude,longitude,bedrooms,bathrooms,squareFootage,price,daysOnMarket,yearBuilt,lotSize
count,3593.000000,3593.000000,3514.000000,3539.000000,3072.000000,3.593000e+03,3063.000000,1006.000000,2.530000e+02
mean,38.048722,-78.489303,2.589357,2.005651,1545.739583,3.003771e+03,131.890957,1981.985089,3.692683e+04
std,0.038130,0.043470,1.087993,0.910554,800.212154,6.670515e+04,200.155308,31.023226,1.363673e+05
min,37.895933,-78.761071,0.000000,1.000000,200.000000,2.000000e+01,1.000000,1800.000000,4.360000e+02
25%,38.024491,-78.507789,2.000000,1.000000,972.000000,1.395000e+03,16.000000,1964.000000,3.920000e+03
50%,38.036166,-78.492022,3.000000,2.000000,1350.000000,1.750000e+03,43.000000,1986.000000,7.405000e+03
75%,38.067244,-78.462647,3.000000,2.500000,2000.000000,2.200000e+03,137.000000,2007.000000,1.263200e+04
max,38.199458,-78.334474,10.000000,6.000000,7568.000000,4.000000e+06,1728.000000,2121.000000,1.393920e+06


In [4]:
ltr_df.dtypes

id                   object
formattedAddress     object
addressLine1         object
addressLine2         object
city                 object
state                object
zipCode              object
county               object
latitude            float64
longitude           float64
propertyType         object
bedrooms            float64
bathrooms           float64
squareFootage       float64
status               object
price                 int64
listedDate           object
removedDate          object
createdDate          object
lastSeenDate         object
daysOnMarket        float64
yearBuilt           float64
lotSize             float64
dtype: object

In [5]:
ltr_df.isnull().sum()

id                     0
formattedAddress       2
addressLine1           0
addressLine2        2356
city                   0
state                  0
zipCode                2
county                35
latitude               0
longitude              0
propertyType           4
bedrooms              79
bathrooms             54
squareFootage        521
status                 0
price                  0
listedDate           530
removedDate          180
createdDate            0
lastSeenDate           0
daysOnMarket         530
yearBuilt           2587
lotSize             3340
dtype: int64

In [6]:
ltr_df.to_csv('../data/cville_long_term_rentals.csv', index=False)

### Properties

- The following cell will pull all properties in Charlottesville. It results in 39,245 properties.
- API docs: https://developers.rentcast.io/reference/property-records
- Property types: https://developers.rentcast.io/reference/property-types

In [9]:
properties_endpoint = "/properties"

properties = RentCastAPIClient.create(
    endpoint=properties_endpoint, 
    query_params=query_params, 
    api_key=RENTCAST_API_KEY, 
    limit=500
)

properties_df = properties.to_frame()

INFO:rentradar.ingest.rentcast_client:Starting data fetch for endpoint: /properties
INFO:rentradar.ingest.rentcast_client:Completed fetching all data for endpoint: /properties


In [15]:
properties_df.describe()

,latitude,longitude,lastSalePrice,bedrooms,bathrooms,squareFootage,lotSize,yearBuilt
count,39245.000000,39245.000000,1.206700e+04,26756.000000,26724.000000,26636.000000,2.003400e+04,24239.000000
mean,38.047849,-78.485840,1.263731e+06,3.146584,2.431616,1834.032024,8.101704e+04,1980.733529
std,0.040759,0.043238,6.368437e+06,1.023607,0.987986,4269.540872,9.461037e+05,30.382441
min,37.796512,-78.732894,8.500000e+02,0.000000,1.000000,200.000000,1.000000e+00,1730.000000
25%,38.024046,-78.503983,1.869625e+05,3.000000,2.000000,1164.000000,4.356000e+03,1965.000000
50%,38.037984,-78.485854,2.850000e+05,3.000000,2.500000,1565.500000,8.059000e+03,1985.000000
75%,38.069459,-78.461811,4.079080e+05,4.000000,3.000000,2148.000000,1.481000e+04,2003.000000
max,38.243089,-78.152350,1.210981e+08,20.000000,15.000000,336000.000000,4.351644e+07,2121.000000


In [14]:
properties_df.isnull().sum()

id                      0
formattedAddress        0
addressLine1            0
addressLine2        28290
city                    0
state                   0
zipCode                 0
county                  2
latitude                0
longitude               0
features             4856
lastSaleDate        27179
lastSalePrice       27178
bedrooms            12489
bathrooms           12521
squareFootage       12609
propertyType        11801
lotSize             19211
yearBuilt           15006
assessorID          16964
legalDescription    17731
subdivision         18184
zoning              17394
taxAssessments      17024
propertyTaxes       17543
owner               16999
ownerOccupied       16999
dtype: int64

In [16]:
properties_df.to_csv('../data/cville_properties.csv', index=False)

### Sale Listings

- The following cell will pull all sales listings with the price in Charlottesville. It results in 6005 listings.
- API docs: https://developers.rentcast.io/reference/sale-listings

In [17]:
sale_listings_endpoint = "/listings/sale"

sl = RentCastAPIClient.create(
    endpoint=sale_listings_endpoint, 
    query_params=query_params, 
    api_key=RENTCAST_API_KEY, 
    limit=500
)

sl_df = sl.to_frame()

INFO:rentradar.ingest.rentcast_client:Starting data fetch for endpoint: /listings/sale
INFO:rentradar.ingest.rentcast_client:Completed fetching all data for endpoint: /listings/sale


In [18]:
sl_df.describe()

,latitude,longitude,bedrooms,bathrooms,squareFootage,lotSize,yearBuilt,price,daysOnMarket
count,6005.000000,6005.000000,5648.000000,5444.000000,5654.000000,2.565000e+03,2795.000000,6.005000e+03,6005.000000
mean,38.060375,-78.489824,3.383676,2.787656,2374.554652,1.304301e+05,1994.269767,5.648548e+05,249.491590
std,0.051770,0.064215,0.993656,0.981178,1219.263286,9.396106e+05,29.934494,5.669050e+05,246.506567
min,37.812802,-78.761071,0.000000,1.000000,384.000000,4.360000e+02,1730.000000,5.000000e+03,1.000000
25%,38.024774,-78.511774,3.000000,2.000000,1643.000000,4.356000e+03,1975.000000,3.300000e+05,27.000000
50%,38.059013,-78.478488,3.000000,2.500000,2155.000000,7.405000e+03,2001.000000,4.400000e+05,148.000000
75%,38.100422,-78.444930,4.000000,3.500000,2795.000000,2.178000e+04,2022.000000,6.249000e+05,453.000000
max,38.243089,-78.152350,16.000000,11.500000,17647.000000,2.539548e+07,2024.000000,1.200000e+07,2259.000000


In [19]:
sl_df.isnull().sum()

id                     0
formattedAddress       0
addressLine1           0
addressLine2        4975
city                   0
state                  0
zipCode                0
county                 0
latitude               0
longitude              0
propertyType          11
bedrooms             357
bathrooms            561
squareFootage        351
lotSize             3440
yearBuilt           3210
status                 0
price                  0
listedDate             0
removedDate          220
createdDate            0
lastSeenDate           0
daysOnMarket           0
dtype: int64

In [20]:
sl_df.to_csv('../data/cville_sale_listings.csv', index=False)

### Market Statistics

- The following endpoint returns aggregate rental listing data, averages, listing statistics, and historical trends for a single zip code.
- API docs: https://developers.rentcast.io/reference/market-statistics

In [2]:
import pandas as pd

rc_client = RentCastAPIClient(api_key=RENTCAST_API_KEY)

properties_df = pd.read_csv('../data/cville_properties.csv')

zipcodes = properties_df.zipCode.unique()

query_params = {"zipCodes": zipcodes, "historyRange": 36}

current_stats, historical_stats = rc_client.process_markets_endpoint(
            endpoint="/markets",
            query_params=query_params
        )

ERROR:rentradar.ingest.rentcast_client:Failed to fetch data for zip code: 22904
ERROR:rentradar.ingest.rentcast_client:Failed to fetch data for zip code: 22906


In [3]:
current_stats.describe()

,bedrooms,averageRent,minRent,maxRent,totalListings,zipCode
count,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000
mean,2.800000,2281.288889,1890.511111,3020.222222,9.866667,22998.444444
std,1.603972,968.990140,1030.943712,1410.510850,13.610825,347.771905
min,0.000000,750.000000,610.000000,750.000000,1.000000,22901.000000
25%,2.000000,1525.000000,1250.000000,1750.000000,1.000000,22903.000000
50%,3.000000,2260.000000,1650.000000,2850.000000,4.000000,22923.000000
75%,4.000000,2681.000000,2400.000000,3600.000000,11.000000,22942.000000
max,8.000000,5450.000000,5450.000000,6500.000000,60.000000,24590.000000


In [4]:
current_stats.head()

,bedrooms,averageRent,minRent,maxRent,totalListings,lastUpdatedDate,zipCode
0,1,1439,610,3000,35,2024-02-24T00:00:00.000Z,22903
1,2,1734,875,2850,60,2024-02-24T00:00:00.000Z,22903
2,3,2260,1425,4000,46,2024-02-24T00:00:00.000Z,22903
3,4,2667,690,5800,20,2024-02-24T00:00:00.000Z,22903
4,5,3800,3000,4700,7,2024-02-24T00:00:00.000Z,22903


In [5]:
historical_stats.describe()

,bedrooms,averageRent,minRent,maxRent,totalListings,zipCode
count,1706.000000,1706.000000,1706.000000,1706.000000,1706.000000,1706.000000
mean,2.783118,2184.987403,1759.187573,2789.412661,10.430832,22986.381594
std,1.829981,1188.057205,1196.744356,1514.572759,15.693328,322.455422
min,0.000000,675.000000,494.000000,675.000000,1.000000,22901.000000
25%,1.000000,1381.500000,995.000000,1700.000000,1.000000,22903.000000
50%,3.000000,1911.030000,1409.500000,2475.000000,4.000000,22911.000000
75%,4.000000,2582.500000,2200.000000,3500.000000,12.000000,22942.000000
max,10.000000,8325.000000,8325.000000,10000.000000,105.000000,24590.000000


In [6]:
historical_stats.head()

,bedrooms,averageRent,minRent,maxRent,totalListings,date,zipCode
0,0,916.67,515,1299,3,2021-03,22903
1,1,1148.33,600,1560,12,2021-03,22903
2,2,1653.70,950,4000,10,2021-03,22903
3,3,1664.00,494,2700,11,2021-03,22903
4,4,1142.50,585,1700,2,2021-03,22903


In [7]:
current_stats.to_csv('../data/cville_current_market_stats.csv', index=False)

In [8]:
historical_stats.to_csv('../data/cville_historical_market_stats.csv', index=False)